# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
#with open("./config.yaml", 'rb') as f:
#    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/DAGMM/add_metaAE/latent5_mixture3/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [9]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
                window_size=config['preprocessing']['window_size'],
                hop_size=config['preprocessing']['hop_size'],
                mel_bins=config['preprocessing']['mel_bins'],
                fmin=config['preprocessing']['fmin'],
                fmax=config['preprocessing']['fmax'],
                latent_size=config['fit']['latent_size'],
                mixture_size=config['fit']['mixture_size'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['fit']['num_epochs']
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    criterion = nn.MSELoss()
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [10]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [ ]:
for machine_type in machine_types:
    history = run(machine_type)
    #with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    #    pickle.dump(history , file)

2020-11-28 18:50:36,209 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-28 18:50:36,210 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-28 18:50:37,164 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 15/15 [00:01<00:00,  9.34it/s]
2020-11-28 18:50:54,186 - pytorch_modeler.py - INFO - Epoch 1/300:train_loss:251.959172, tr_rec:17.367813, tr_eng:694.889853, tr_covd:1085.180861, tr_metaloss:164.993855, val_AUC:0.548089, val_pAUC:0.525519
100%|██████████| 15/15 [00:01<00:00, 10.65it/s]
2020-11-28 18:51:07,162 - pytorch_modeler.py - INFO - Epoch 2/300:train_loss:190.285161, tr_rec:16.205053, tr_eng:694.686913, tr_covd:1084.321392, tr_metaloss:104.502982, val_AUC:0.502303, val_pAUC:0.490397
100%|██████████| 15/15 [00:01<00:00, 10.42it/s]
2020-11-28 18:51:20,233 - pytorch_modeler.py - INFO - Epoch 3/300:train_loss:100.263556, tr_rec:6.933050, tr_eng:689.989995, tr_covd:1087.600698, tr_metaloss:24.222744, val_AUC:0.502754, val_pAUC:0.505308
100%|██████████| 15/15 [00:01<00:00, 10.38it/s]
2020-11-28 18:51:33,426 - pytorch_modeler.py - INFO - Epoch 4/300:train_loss:74.898704, tr_rec:2.280000, tr_eng:641.205750, tr_covd:1153.874971, tr_metaloss:8.382739, val_AUC:0.498286, val_

100%|██████████| 15/15 [00:02<00:00,  5.79it/s]
2020-11-28 19:00:25,380 - pytorch_modeler.py - INFO - Epoch 29/300:train_loss:0.377837, tr_rec:0.157925, tr_eng:-11.864259, tr_covd:1238.404121, tr_metaloss:1.282497, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 15/15 [00:02<00:00,  5.97it/s]
2020-11-28 19:00:47,915 - pytorch_modeler.py - INFO - Epoch 30/300:train_loss:-4.144995, tr_rec:0.155745, tr_eng:-53.274900, tr_covd:1218.478672, tr_metaloss:0.904902, val_AUC:0.499644, val_pAUC:0.499981
100%|██████████| 15/15 [00:02<00:00,  5.90it/s]
2020-11-28 19:01:11,471 - pytorch_modeler.py - INFO - Epoch 31/300:train_loss:3.311685, tr_rec:0.153325, tr_eng:22.096446, tr_covd:1224.298088, tr_metaloss:0.826285, val_AUC:0.500000, val_pAUC:0.500000
100%|██████████| 15/15 [00:02<00:00,  5.97it/s]
2020-11-28 19:01:35,531 - pytorch_modeler.py - INFO - Epoch 32/300:train_loss:3.391842, tr_rec:0.164615, tr_eng:18.533601, tr_covd:1265.427977, tr_metaloss:1.247324, val_AUC:0.500893, val_pAUC:0.5000

history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)